In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import numpy as np
import os
import torch

import torch.nn as nn
import torch.optim as optim

from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import EfficientNet

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/efficientnet/models//'

# CONFIG VARIABLES
DATA_DIR = 'E:/Datasets/cor-splits/sgkf-8-1-1-2000'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 50
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0
MIN_EPOCH = 20

In [4]:
working_folder = 'notebooks/efficientnet'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [5]:
# # efficientnet_b0
# mean = np.array([0.485, 0.456, 0.406])
# std = np.array([0.229, 0.224, 0.225])

# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomHorizontalFlip(),
#         transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(), # When converting to tensor, pytorch automatically rescales to [0,1]
#         transforms.Normalize(mean, std)
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std)
#     ]),
# }

In [6]:
# model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

# for param in model.parameters(): # Turning parameters not trainable
#     param.requires_grad = True

# # A newly defined layer is created with requires_grad=True by default
# model.classifier = nn.Sequential(
#     nn.Dropout(p=0.2, inplace=True),
#     nn.Linear(in_features=1280, out_features=2)
#     )

# if CONTINUE_TRAINING:
#     output_folder = MODEL_WEIGHTS_PATH
#     model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
# else:
#     if not os.path.exists(output_folder): 
#         os.makedirs(output_folder)

In [7]:
eff_net = EfficientNet()

if CONTINUE_TRAINING:
    model = eff_net.load(MODEL_WEIGHTS_PATH)
else:
    model = eff_net.load()

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
             
model = model.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()

# Only the fully connected layer is being optimized
# optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=LR)
optimizer = optim.Adamax(model.parameters(), lr=LR)

# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)

In [9]:
dataset = PytorchDataset(DATA_DIR, eff_net.data_transforms)
trainer = PytorchTraining(device, dataset, output_folder)
model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/50
----------
train Loss: 0.1753 Acc: 0.9315
val Loss: 0.4168 Acc: 0.8169
Epoch duration: 0 m 52s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 2/50
----------
train Loss: 0.0592 Acc: 0.9784
val Loss: 0.5673 Acc: 0.7934
Epoch duration: 0 m 51s
Learning Rate = 0.001
Loss didnt decrease from 0.4168. Increasing patience counter

Epoch 3/50
----------
train Loss: 0.0328 Acc: 0.9890
val Loss: 0.6002 Acc: 0.7981
Epoch duration: 0 m 49s
Learning Rate = 0.001
Loss didnt decrease from 0.4168. Increasing patience counter

Epoch 4/50
----------
train Loss: 0.0243 Acc: 0.9920
val Loss: 0.5349 Acc: 0.8169
Epoch duration: 0 m 46s
Learning Rate = 0.001
Loss didnt decrease from 0.4168. Increasing patience counter

Epoch 5/50
----------
train Loss: 0.0162 Acc: 0.9947
val Loss: 0.5289 Acc: 0.8192
Epoch duration: 0 m 52s
Learning Rate = 0.001
Loss didnt decrease from 0.4168. Increasing patience counter

Epoch 6/50
----------
train Loss: 0.0145 Acc: 0.9953
val Loss: 0.4